In [ ]:
from __future__ import print_function
from __future__ import division

import networkx as nx
import numpy as np
import cv2
import re
import os
import sys
import math
import utils
import tensorflow as tf
import keras
import segmentation
import time

import BachNet
import ChopinNet
import utils
import shutil
from keras.models import load_model
from keras import backend as K


from sklearn import feature_extraction
import matplotlib.pyplot as plt

In [ ]:
train_path = "data/train/input/"
test_path = "data/test/input/"

output_path = "output"

path_to_original_images = "original/"
path_to_gt_images = "gt/"

receptive_field_shape = (23, 23)

bach = BachNet.BachNet()
chopin = ChopinNet.Chopin()

# Training

## Bach

In [ ]:
bach.build(receptive_field_shape[0], receptive_field_shape[1], 1)
bach.load_model()

In [ ]:
X = []
Y = []

originals = os.listdir(os.path.join(train_path))

for i, filename in enumerate(originals):
    
    if "gt" in filename.split("_"):
        continue
    
    gt_filename = filename + "_gt"
    
    image_path = os.path.join(train_path, filename)
    gt_path = os.path.join(train_path, gt_filename)
    
    img = cv2.imread(image_path, 0)
    gt = cv2.imread(gt_path, 0)
    
    if gt is None:
        continue
    
    x = utils.prepare_input_images(img, width=receptive_field_shape[0],
                                   height=receptive_field_shape[1])
    
    X.append(x)

    gt = cv2.imread(gt_path, 0)
    y = np.zeros_like(gt)
    y[gt == 255] = 1
    y = y.flatten()
    
    Y.append(y)
    
X = np.concatenate(X)
Y = np.concatenate(Y)

In [ ]:
bach.model.fit(X, Y, batch_size=1000, epochs=32)

In [ ]:
originals = os.listdir(os.path.join(train_path))

for i, filename in enumerate(originals):
    
    if "gt" in filename.split("_"):
        continue
    
    image_path = os.path.join(train_path, filename)
    
    img = cv2.imread(image_path, 0)

    bach_filename = "data/train/bach/" + filename

    boundary_probabilities = bach.boundary_probabilities(img, batch_size=3000, verbose=1)

    plt.imsave(bach_filename, boundary_probabilities, cmap='gray')

    bach.model.save('saved_model/Bach/model.h5')

## Chopin

In [ ]:
originals = os.listdir(os.path.join(train_path))

number_files = len(originals)

progress = 0

for i, filename in enumerate(originals):
    
    if "gt" in filename.split("_"):
        continue
        
    sys.stdout.write("\nTotal Progress: [%d%%]\n" % progress)
    progress = i + 1 / number_files * 100
    
    gt_filename = filename + "_gt"
    
    image_path = os.path.join(train_path, filename)
    gt_path = os.path.join(train_path, gt_filename)
    
    print("Now training on", filename)
    print("---------------------------\n")
    
    img = cv2.imread(image_path, 0)
    gt = cv2.imread(gt_path, 0)
    
    foldername = "data/train/chopin/" + filename
    if os.path.exists(foldername):
        shutil.rmtree(foldername)
    
    os.makedirs(foldername)
        
    os.system("gmic -v -1 data/train/input/" + gt_filename + " -channels 1 -threshold 10% -negative -label_fg 0,0 -o -.asc | tail -n +2 | awk '{ for (i = 1; i<=NF; i++) {x[$i] += i; y[$i] += NR; n[$i]++; } } END { for (v in x) print x[v]/n[v],y[v]/n[v] }' > " + foldername + "/seeds.txt")
    
    seeds = []
    f = open(foldername + "/seeds.txt", 'r')
    for line in f:
        y = int(float(re.split(' ', line)[0]))
        x = int(float(re.split(' ', line)[1]))
        seed = (x - 1, y - 1)
        seeds.append(seed)

    seeds = seeds[1:]
    
    
    boundary_probabilties, segmentations, loss_timeline, accuracy_timeline = ChopinNet.fit(img,
                                                                                            gt,
                                                                                            seeds,
                                                                                            num_epochs=32)
    plt.plot(loss_timeline)
    plt.savefig(os.path.join(foldername, "loss.png"))
    plt.clf()   # Clear figure
    
    for i, seg in enumerate(segmentations):
        mask = utils.transparent_mask(img, seg, alpha=0.7)
        epoch = "Epoch " + str(i + 1)
        plt.imsave(os.path.join(foldername, epoch), mask)
    
sys.stdout.write("\nDone!: [100%]\n")

# Testing

## Bach

In [ ]:
originals = os.listdir(os.path.join(test_path))

number_files = len(originals)

progress = 0

for i, filename in enumerate(originals):
    
    if "gt" in filename.split("_"):
        continue
        
    sys.stdout.write("\nTotal Progress: [%d%%]\n" % progress)
    progress = i + 1 / number_files * 100
    
    gt_filename = filename + "_gt"
    
    image_path = os.path.join(test_path, filename)
    gt_path = os.path.join(test_path, gt_filename)
    
    print("Now testing on", filename)
    print("---------------------------\n")
    
    img = cv2.imread(image_path, 0)
    x = utils.prepare_input_images(img)
    
    b_probs = b_net.boundary_probabilities(img, batch_size=3000, verbose=1)
    
    bach_filename = "data/test/bach/" + filename
    plt.imsave(bach_filename, b_probs, cmap='gray')
    
sys.stdout.write("\nDone!: [100%]\n")

## Chopin

In [ ]:
originals = os.listdir(os.path.join(test_path))

number_files = len(originals)

progress = 0

for i, filename in enumerate(originals):
    
    if "gt" in filename.split("_"):
        continue
        
    sys.stdout.write("\nTotal Progress: [%d%%]\n" % progress)
    progress = i + 1 / number_files * 100
    
    gt_filename = filename + "_gt"
    
    image_path = os.path.join(test_path, filename)
    gt_path = os.path.join(test_path, gt_filename)
    
    print("Now training on", filename)
    print("---------------------------\n")
    
    img = cv2.imread(image_path, 0)
    gt = cv2.imread(gt_path, 0)
    
    foldername = "data/test/chopin/" + filename
    if os.path.exists(foldername):
        shutil.rmtree(foldername)
    
    os.makedirs(foldername)
        
    os.system("gmic -v -1 data/test/input/" + gt_filename + " -channels 1 -threshold 10% -negative -label_fg 0,0 -o -.asc | tail -n +2 | awk '{ for (i = 1; i<=NF; i++) {x[$i] += i; y[$i] += NR; n[$i]++; } } END { for (v in x) print x[v]/n[v],y[v]/n[v] }' > " + foldername + "/seeds.txt")
    
    seeds = []
    f = open(foldername + "/seeds.txt", 'r')
    for line in f:
        y = int(float(re.split(' ', line)[0]))
        x = int(float(re.split(' ', line)[1]))
        seed = (x - 1, y - 1)
        seeds.append(seed)

    seeds = seeds[1:]
    
    seg = ChopinNet.predict(img, seeds)
    
    mask = utils.transparent_mask(img, seg, alpha=0.5)
    plt.imsave(os.path.join(foldername, filename), mask)